<a href="https://colab.research.google.com/github/gaobowen/machine-learning/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transformer中英翻译

## 1. 数据预处理

### 1.1 下载数据集

In [65]:
!wget -O en-zh.tsv "https://tatoeba.org/zh-cn/exports/download/31726/%E6%9C%89%E8%8B%B1%E8%AF%AD-%E4%B8%AD%E6%96%87%E6%99%AE%E9%80%9A%E8%AF%9D%E5%AF%B9%E5%BA%94%E5%8F%A5%20-%202023-02-08.tsv"

--2023-02-10 10:14:29--  https://tatoeba.org/zh-cn/exports/download/31726/%E6%9C%89%E8%8B%B1%E8%AF%AD-%E4%B8%AD%E6%96%87%E6%99%AE%E9%80%9A%E8%AF%9D%E5%AF%B9%E5%BA%94%E5%8F%A5%20-%202023-02-08.tsv
Resolving tatoeba.org (tatoeba.org)... 94.130.77.194, 2a01:4f8:c0c:2b50::1
Connecting to tatoeba.org (tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5667844 (5.4M) [application/octet-stream]
Saving to: ‘en-zh.tsv’

en-zh.tsv           100%[===================>]   5.41M  5.05MB/s    in 1.1s    

2023-02-10 10:14:31 (5.05 MB/s) - ‘en-zh.tsv’ saved [5667844/5667844]



In [66]:
import torch
import numpy as np
import pandas as pd

all_data = pd.read_csv('en-zh.tsv',sep='\t',on_bad_lines='skip',names=['NO.1','en','NO.2','zh'])
print(type(all_data))
print(type(all_data.values))
all_data

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


,NO.1,en,NO.2,zh
0,1277,I have to go to sleep.,2,我该去睡觉了。
1,1280,Today is June 18th and it is Muiriel's birthday!,5,今天是６月１８号，也是Muiriel的生日！
2,1282,Muiriel is 20 now.,7,Muiriel现在20岁了。
3,1283,"The password is ""Muiriel"".",8,"密码是""Muiriel""。"
4,1284,I will be back soon.,9,我很快就會回來。
...,...,...,...,...
66791,2458877,He and I are friends.,6149234,我們是朋友。
66792,2458877,He and I are friends.,5624940,我们是朋友。
66793,11516479,I'm drooling.,11516485,我在流口水。
66794,2684425,Is he a friend?,11518464,他是一个朋友吗？


### 1.2 繁体转成简体 全角数字转为半角

In [67]:
#将繁体转成简体
!pip install zhconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
from zhconv import convert
convert('我很快就會回來', 'zh-cn')

'我很快就会回来'

In [69]:
def Q2B(uchar):
  """判断一个unicode是否是全角数字"""
  if uchar >= u'\uff10' and uchar <= u'\uff19':
    """单个字符 全角转半角"""
    inside_code = ord(uchar)
    if inside_code == 0x3000:
        inside_code = 0x0020
    else:
        inside_code -= 0xfee0
    if inside_code < 0x0020 or inside_code > 0x7e: #转完之后不是半角字符返回原来的字符
        return uchar
    return chr(inside_code)
  else:
    return uchar

def stringpartQ2B(ustring):
  return "".join([Q2B(uchar) for uchar in ustring])


def convertSimple(x):
  return stringpartQ2B(convert(x.values[0], 'zh-cn'))


zh_data = all_data.iloc[:,[3]].apply(convertSimple, axis=1).rename('zhs',inplace=True)
 
all_data = pd.concat([all_data.iloc[:,[1]], zh_data], axis=1)

all_data = all_data.applymap(lambda x: x.lower()) #英文全部转为小写

all_data

,en,zhs
0,i have to go to sleep.,我该去睡觉了。
1,today is june 18th and it is muiriel's birthday!,今天是6月18号，也是muiriel的生日！
2,muiriel is 20 now.,muiriel现在20岁了。
3,"the password is ""muiriel"".","密码是""muiriel""。"
4,i will be back soon.,我很快就会回来。
...,...,...
66791,he and i are friends.,我们是朋友。
66792,he and i are friends.,我们是朋友。
66793,i'm drooling.,我在流口水。
66794,is he a friend?,他是一个朋友吗？


### 1.3 中英文进行分词

In [70]:
!pip install nltk #英文分词
!pip install jieba #中文分词

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import nltk
nltk.download('punkt')
nltk.word_tokenize("today is june 18th and it is muiriel's birthday!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['today',
 'is',
 'june',
 '18th',
 'and',
 'it',
 'is',
 'muiriel',
 "'s",
 'birthday',
 '!']

In [72]:
import jieba
jieba.lcut("今天是6月18号，也是muiriel的生日！")

['今天', '是', '6', '月', '18', '号', '，', '也', '是', 'muiriel', '的', '生日', '！']

In [73]:
en_list = all_data.iloc[:,[0]].applymap(lambda x: nltk.word_tokenize(x))
zhs_list = all_data.iloc[:,[1]].applymap(lambda x: jieba.lcut(x))
all_data = pd.concat([en_list,zhs_list],axis=1)
all_data

,en,zhs
0,"[i, have, to, go, to, sleep, .]","[我, 该, 去, 睡觉, 了, 。]"
1,"[today, is, june, 18th, and, it, is, muiriel, ...","[今天, 是, 6, 月, 18, 号, ，, 也, 是, muiriel, 的, 生日, ！]"
2,"[muiriel, is, 20, now, .]","[muiriel, 现在, 20, 岁, 了, 。]"
3,"[the, password, is, ``, muiriel, '', .]","[密码, 是, "", muiriel, "", 。]"
4,"[i, will, be, back, soon, .]","[我, 很快, 就, 会, 回来, 。]"
...,...,...
66791,"[he, and, i, are, friends, .]","[我们, 是, 朋友, 。]"
66792,"[he, and, i, are, friends, .]","[我们, 是, 朋友, 。]"
66793,"[i, 'm, drooling, .]","[我, 在, 流口水, 。]"
66794,"[is, he, a, friend, ?]","[他, 是, 一个, 朋友, 吗, ？]"


In [74]:
# 添加两个特殊单词 BOF EOF 分别表示 句子的开始和结束  
all_data = all_data.applymap(lambda x: ['BOF']+ x +['EOF'])
all_data

,en,zhs
0,"[BOF, i, have, to, go, to, sleep, ., EOF]","[BOF, 我, 该, 去, 睡觉, 了, 。, EOF]"
1,"[BOF, today, is, june, 18th, and, it, is, muir...","[BOF, 今天, 是, 6, 月, 18, 号, ，, 也, 是, muiriel, 的,..."
2,"[BOF, muiriel, is, 20, now, ., EOF]","[BOF, muiriel, 现在, 20, 岁, 了, 。, EOF]"
3,"[BOF, the, password, is, ``, muiriel, '', ., EOF]","[BOF, 密码, 是, "", muiriel, "", 。, EOF]"
4,"[BOF, i, will, be, back, soon, ., EOF]","[BOF, 我, 很快, 就, 会, 回来, 。, EOF]"
...,...,...
66791,"[BOF, he, and, i, are, friends, ., EOF]","[BOF, 我们, 是, 朋友, 。, EOF]"
66792,"[BOF, he, and, i, are, friends, ., EOF]","[BOF, 我们, 是, 朋友, 。, EOF]"
66793,"[BOF, i, 'm, drooling, ., EOF]","[BOF, 我, 在, 流口水, 。, EOF]"
66794,"[BOF, is, he, a, friend, ?, EOF]","[BOF, 他, 是, 一个, 朋友, 吗, ？, EOF]"


### 1.4 制作中英文词典

统计单词的频率，降序排列，用单词的下标作为单词的id

In [75]:
from collections import Counter

c = Counter(['a', 'b', 'b', 'a', 'a', 'e', 'd'])
print(c)
c += Counter(['a','a','b','b'])
print(c.most_common())
for idx,ele in enumerate(c.most_common(), start=2):
  print(idx, ele)

Counter({'a': 3, 'b': 2, 'e': 1, 'd': 1})
[('a', 5), ('b', 4), ('e', 1), ('d', 1)]
2 ('a', 5)
3 ('b', 4)
4 ('e', 1)
5 ('d', 1)


In [76]:
#统计单词的频率
en_table = Counter([])
zhs_table = Counter([])

for row in all_data.values:
  en_table.update(row[0])
  zhs_table.update(row[1])

print(en_table.most_common(10))
print(zhs_table.most_common(10))


[('BOF', 66796), ('EOF', 66796), ('.', 56946), ('the', 18887), ('i', 18253), ('to', 14170), ('you', 12703), ('is', 11677), ('a', 11418), ('?', 9728)]
[('BOF', 66796), ('EOF', 66796), ('。', 55871), ('我', 21847), ('的', 21742), ('了', 14106), ('你', 12079), ('他', 9372), ('，', 8954), ('？', 8748)]


创建 单词到id 和 id到单词 的映射字典

In [77]:
en_to_id = {"UNK": 0, "PAD": 1} # UNK表示未知字符，PAD表示占位符
zhs_to_id = {"UNK": 0, "PAD": 1}

id_to_en = {0: 'UNK', 1:'PAD'}
id_to_zhs = {0: 'UNK', 1:'PAD'}

#用数组下标作为单词的id
for idx,ele in enumerate(en_table.most_common(50000), start=2):
  en_to_id[ele[0]] = idx
  id_to_en[idx] = ele[0]

print(en_to_id)
print(id_to_en)
print("=========")

for idx,ele in enumerate(zhs_table.most_common(50000), start=2):
  zhs_to_id[ele[0]] = idx
  id_to_zhs[idx] = ele[0]

print(zhs_to_id)
print(id_to_zhs)

{'UNK': 0, 'PAD': 1, 'BOF': 2, 'EOF': 3, '.': 4, 'the': 5, 'i': 6, 'to': 7, 'you': 8, 'is': 9, 'a': 10, '?': 11, ',': 12, 'he': 13, "n't": 14, 'in': 15, 'do': 16, 'of': 17, 'it': 18, "'s": 19, 'tom': 20, 'that': 21, 'this': 22, 'my': 23, 'have': 24, 'are': 25, 'she': 26, 'was': 27, 'me': 28, 'for': 29, 'and': 30, 'we': 31, 'not': 32, 'what': 33, 'on': 34, 'your': 35, 'his': 36, 'at': 37, 'be': 38, 'with': 39, "'m": 40, 'like': 41, 'can': 42, 'did': 43, 'there': 44, 'has': 45, 'her': 46, '!': 47, 'will': 48, 'as': 49, 'go': 50, 'how': 51, 'they': 52, 'very': 53, 'know': 54, 'him': 55, 'want': 56, 'all': 57, 'from': 58, 'time': 59, 'an': 60, 'no': 61, 'up': 62, 'when': 63, 'by': 64, 'if': 65, "'ll": 66, 'does': 67, 'here': 68, 'one': 69, 'good': 70, 'mary': 71, 'had': 72, 'about': 73, "'re": 74, 'but': 75, 'out': 76, "'ve": 77, 'think': 78, 'am': 79, 'would': 80, 'so': 81, 'please': 82, 'ca': 83, 'come': 84, 'get': 85, 'should': 86, 'who': 87, 'than': 88, 'going': 89, 'people': 90, 'been

把翻译数据全部转成id的表示形式

In [78]:
all_en_id = all_data.iloc[:,[0]].applymap(lambda x: [en_to_id.get(word, 0) for word in x]) # UNK=0
all_zhs_id = all_data.iloc[:,[1]].applymap(lambda x: [zhs_to_id.get(word, 0) for word in x]) # UNK=0
all_data = pd.concat([all_en_id, all_zhs_id], axis=1)
all_data

,en,zhs
0,"[2, 6, 24, 7, 50, 7, 336, 4, 3]","[2, 5, 166, 22, 242, 7, 4, 3]"
1,"[2, 118, 9, 1675, 5486, 30, 18, 9, 2944, 19, 5...","[2, 76, 12, 1462, 370, 2491, 992, 10, 48, 12, ..."
2,"[2, 2944, 9, 1271, 95, 4, 3]","[2, 3502, 66, 1513, 570, 7, 4, 3]"
3,"[2, 5, 2239, 9, 137, 2944, 140, 4, 3]","[2, 2233, 12, 305, 3502, 305, 4, 3]"
4,"[2, 6, 48, 38, 142, 212, 4, 3]","[2, 5, 250, 39, 28, 167, 4, 3]"
...,...,...
66791,"[2, 13, 30, 6, 25, 193, 4, 3]","[2, 17, 12, 93, 4, 3]"
66792,"[2, 13, 30, 6, 25, 193, 4, 3]","[2, 17, 12, 93, 4, 3]"
66793,"[2, 6, 40, 14601, 4, 3]","[2, 5, 13, 12649, 4, 3]"
66794,"[2, 9, 13, 10, 209, 11, 3]","[2, 9, 12, 26, 93, 16, 11, 3]"
